# Enriquecimento dos Dados

Nesse arquivo serão gerados os seguintes enriquecimentos de para cada noticia do dataset da Folha de São Paulo

- categories
- concepts
- keywords
- sentiment
- entities
- classification


In [11]:
import pandas as pd
import os
import json
from tqdm import tqdm

## Carrega e limpa dataset noticias Folha

link do dataset: https://www.kaggle.com/marlesson/news-of-the-site-folhauol

conferir nome do arquivo


In [12]:
news = pd.read_csv('news.csv')

news = news.dropna(subset=['text', 'title', 'date', 'category'])
news = news[news["title"] != "Aviso"]
news = news[news["title"] != "Aviso de férias"]
news = news[news["text"].str.len() >= 60]
news.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


## Login NLU

In [6]:
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import * 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

apikey = 'HhNXvrfHCVgWjgCSlcAIzi-fW8_zoCyCI2FS6dl_dcMn'
url = 'https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/6f5ec2bb-44da-4ab6-babf-8a7527f92256'

authenticator = IAMAuthenticator(apikey)
nlu = NaturalLanguageUnderstandingV1(
    version='2021-08-01',
    authenticator=authenticator
)

nlu.set_service_url(url)

## Modelo

Para utilizar a feature de classificação do NLU é necessario enviar os dados de treinamento através da função model_create() e esperar aproximadamente 30 minutos para a conclusão, é possivel observar a etapa atual através da função model_get_status().


In [7]:
def model_create(training_data_filename = 'training_data.json'):
    with open(training_data_filename, 'rb') as file:
        model = nlu.create_classifications_model(language='pt', training_data=file, training_data_content_type='application/json', name='pfe-model', model_version='1.0.1').get_result()
        
        print("Created a NLU Classifications model:")
        print(json.dumps(model, indent=4))

def model_get_status(model_id):
    model_to_view = nlu.get_classifications_model(model_id=model_id).get_result()

    print("Information about the created NLU Classifications model:")
    print(json.dumps(model_to_view, indent=4))

def model_analyze(model_id, text):
    analysis = nlu.analyze(text=text, features=Features(classifications=ClassificationsOptions(model=model_id))).get_result()

    print("Analysis response from trained NLU Classifications model:")
    print(json.dumps(analysis, indent=4))

# Deleta o modelo
def model_delete(model_id):
    deleted_model = nlu.delete_classifications_model(model_id=model_id)
    updated_models_list = nlu.list_classifications_models().get_result()

    print("The NLU Classifications model created in this tutorial has been deleted:")
    print(json.dumps(updated_models_list, indent=4))

def get_model_id():
    res = nlu.list_classifications_models().get_result()
    m_id = res['models'][0]['model_id']
    print(f'Model id: {m_id}')
    return m_id

In [8]:
model_create() # cria modelo

Created a NLU Classifications model:
{
    "name": "pfe-model",
    "user_metadata": null,
    "language": "pt",
    "description": null,
    "model_version": "1.0.1",
    "version": "1.0.1",
    "workspace_id": null,
    "version_description": null,
    "status": "starting",
    "notices": [],
    "model_id": "d5061e6c-9a7c-4295-a6ea-608208f5f9b6",
    "features": [
        "classifications"
    ],
    "created": "2021-11-22T13:52:54Z",
    "last_trained": "2021-11-22T13:52:54Z",
    "last_deployed": null
}


In [9]:
model_id = get_model_id()

Model id: d5061e6c-9a7c-4295-a6ea-608208f5f9b6


In [10]:
model_get_status(model_id) # verifica status 
# model_delete(model_id) # deleta modelo

Information about the created NLU Classifications model:
{
    "name": "pfe-model",
    "user_metadata": null,
    "language": "pt",
    "description": null,
    "model_version": "1.0.1",
    "version": "1.0.1",
    "workspace_id": null,
    "version_description": null,
    "status": "starting",
    "notices": [],
    "model_id": "d5061e6c-9a7c-4295-a6ea-608208f5f9b6",
    "features": [
        "classifications"
    ],
    "created": "2021-11-22T13:52:54Z",
    "last_trained": "2021-11-22T13:52:54Z",
    "last_deployed": null
}


## Passando os dados pelo NLU

Na celula abaixo o texto de cada notícia é enviado para a API do NLU para que possa ser feita a análise. Foi dividido em duas partes por motivos de limitações na quantidade de texto permitida para a análise de classificação do modelo.


In [1]:
%%time

# ######################################## #
# Selecionar range do dataset de noticias  #
# ######################################## #

for i in tqdm(range(0, 100_000)):    
    title = news.values[i][0]
    text = news.values[i][1]
    date = news.values[i][2]
    category = news.values[i][3]
    subcategory = news.values[i][4]
    link = news.values[i][5]
        
    ##  Classificação modelo 
    n = 1500 # chunk length
    chunks = [text[i:i+n] for i in range(0, len(text), n)]
    len_chunks = len(chunks)
    
    error = len_chunks == 1 and len(chunks[0]) < 150
    if error: continue
    
    left = 0
    right = 0
        
    for part in chunks:
        if len(part) < 150: 
            len_chunks -= 1
            continue
            
        res_classification = nlu.analyze(
            text= str(part),
            language= "pt",
            features= Features(
                classifications=ClassificationsOptions(model=model_id)
            )).get_result()        
        
        for classific in res_classification['classifications']:
            if classific['class_name'] == "Direita":
                right += classific['confidence'] 
            elif classific['class_name'] == "Esquerda":
                left += classific['confidence']
    
    if left > right:
        polaridade = {'class_name': 'Esquerda', 'confidence': left / len_chunks}
    else:
        polaridade = {'class_name': 'Direita', 'confidence': right / len_chunks}
        
    ##  Analise Features do NLU
    response = nlu.analyze(
        text=text,
        language= "pt",
        features=Features(
            categories=CategoriesOptions(limit=3),
            concepts=ConceptsOptions(limit=3),
            keywords=KeywordsOptions(sentiment=True,emotion=False,limit=3),
            sentiment=SentimentOptions(),
            entities=EntitiesOptions(sentiment=True,limit=10),
        )).get_result()
    
    file = {"enrichments": response,
            "classification": polaridade,
            "text": text,
            "title": title,
            "category": category,
            "data": date,
            "link": link
           }
    
    with open(f"dados/news_{i}.json", 'w', encoding="utf-8") as f:
        f = json.dump(file, f, ensure_ascii=False, indent=4)